<a href="https://colab.research.google.com/github/wrenchchat/the-littlest-jupyterhub/blob/main/Google_Cloud_Troubleshooting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can copy and paste from this notebook or enter your project and systme account info in the env variables and run the diagnostics from the notebook. Totally up to. I jest find this format very convenient when I'm trying to figure out what happened six months from now.

Run the diagnostics tool gcpdiag (enable APIs first):

In [ ]:
gcloud --project=MYPROJECT services enable \
    cloudresourcemanager.googleapis.com \
    iam.googleapis.com \
    logging.googleapis.com \
    serviceusage.googleapis.com

In [ ]:
gcpdiag lint --project=PROJECT_ID

I also recommend running these commands. There is some overlap but still worhwhile.

VPC Setup

In [ ]:
gcloud compute shared-vpc enable spark-dev-techops-infra
gcloud compute shared-vpc associated-projects add spark-lambda-experiments --host-project spark-dev-techops-infra

Service Account Roles

In [ ]:
gcloud projects add-iam-policy-binding spark-dev-techops-infra \
    --member="serviceAccount:service-<service-project-number>@gcp-sa-composer.iam.gserviceaccount.com" \
    --role="roles/compute.networkUser"

gcloud projects add-iam-policy-binding spark-dev-techops-infra \
    --member="serviceAccount:service-<service-project-number>@gcp-sa-composer.iam.gserviceaccount.com" \
    --role="roles/iam.serviceAccountUser"

Service Account Permissions

In [ ]:
gcloud projects add-iam-policy-binding spark-lambda-experiments \
    --member="serviceAccount:service-<service-project-number>@gcp-sa-composer.iam.gserviceaccount.com" \
    --role="roles/composer.admin"

gcloud projects add-iam-policy-binding spark-lambda-experiments \
    --member="serviceAccount:service-<service-project-number>@gcp-sa-composer.iam.gserviceaccount.com" \
    --role="roles/iam.serviceAccountUser"

Enable Composer API:

In [ ]:
gcloud services enable composer.googleapis.com --project=spark-dev-techops-infra
gcloud services enable composer.googleapis.com --project=spark-lambda-experiments

Check Network Attachment

In [ ]:
gcloud composer environments create my-environment \
    --location us-central1 \
    --network projects/spark-dev-techops-infra/global/networks/rbk-techops-spark-prod-vpc \
    --subnet projects/spark-dev-techops-infra/regions/us-central1/subnetworks/spark-lambda-experiments

Check IAM policies in both projects to ensure correct roles are applied.

In [ ]:
gcloud projects get-iam-policy spark-dev-techops-infra
gcloud projects get-iam-policy spark-lambda-experiments

Example IAM Policy Bindings

In [ ]:
bindings:
- members:
  - serviceAccount:service-<service-project-number>@gcp-sa-composer.iam.gserviceaccount.com
  role: roles/compute.networkUser
- members:
  - serviceAccount:service-<service-project-number>@gcp-sa-composer.iam.gserviceaccount.com
  role: roles/iam.serviceAccountUser

Cloud Logging Queries

In [ ]:
# Cloud Composer Errors
resource.type="cloud_composer_environment"
resource.labels.environment_name="your-environment-name"
severity>=ERROR
jsonPayload.message:("permission" OR "network" OR "subnet" OR "VPC")

# IAM Permission Issues
resource.type="project"
resource.labels.project_id="spark-lambda-experiments"
protoPayload.serviceName="cloudresourcemanager.googleapis.com"
protoPayload.methodName="SetIamPolicy"
protoPayload.status.code="PERMISSION_DENIED"

# VPC Network Attachment Issues
resource.type="gce_network"
resource.labels.project_id="spark-dev-techops-infra"
severity>=ERROR
jsonPayload.message:("shared VPC" OR "subnet" OR "permission denied")

